In [8]:
import openai
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv()

client = openai.Client()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [9]:
vector_store = client.vector_stores.create(
    name="Tutor de Apostilas"
)    

In [10]:
file = ["files/LLM.pdf"]
file_stream = [open(f, "rb") for f in file]
file_batch = client.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id,
    files=file_stream
)

In [11]:
file_batch.status

'completed'

In [13]:
file_batch.file_counts

FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)

In [ ]:
assistant = client.assistants.create(
    
)